In [2]:
import requests
import pandas as pd

API_BASE = "https://data-dbg.krx.co.kr/svc/apis/sto"

# ✅ KOSPI/KOSDAQ만 (코넥스 제외)
ENDPOINTS = {
    "KOSPI": {
        "daily_trade": "/stk_bydd_trd",        # 유가증권 일별매매정보
        "base_info":  "/stk_isu_base_info",    # 유가증권 종목기본정보
    },
    "KOSDAQ": {
        "daily_trade": "/ksq_bydd_trd",        # 코스닥 일별매매정보
        "base_info":  "/ksq_isu_base_info",    # 코스닥 종목기본정보
    }
}

In [3]:
def _krx_post(auth_key: str, endpoint: str, payload: dict, timeout: int = 30) -> dict:
    url = API_BASE + endpoint
    headers = {
        "AUTH_KEY": auth_key.strip(),          # ✅ 공백/개행 제거 습관
        "Content-Type": "application/json",
        "Accept": "application/json",
    }
    r = requests.post(url, headers=headers, json=payload, timeout=timeout)

    if r.status_code != 200:
        raise RuntimeError(f"KRX API Error: {r.status_code} / {r.text}")

    return r.json()

In [4]:
def fetch_daily_trade(auth_key: str, bas_dd: str, market: str = "KOSPI") -> pd.DataFrame:
    market = market.upper()
    endpoint = ENDPOINTS[market]["daily_trade"]
    data = _krx_post(auth_key, endpoint, {"basDd": bas_dd})
    rows = data.get("OutBlock_1", [])
    df = pd.DataFrame(rows)

    # 숫자형 변환(콤마 제거)
    num_cols = [
        "TDD_CLSPRC", "CMPPREVDD_PRC", "FLUC_RT",
        "TDD_OPNPRC", "TDD_HGPRC", "TDD_LWPRC",
        "ACC_TRDVOL", "ACC_TRDVAL", "MKTCAP", "LIST_SHRS"
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", ""), errors="coerce")

    rename_map = {
        "BAS_DD": "Date",
        "ISU_CD": "Code",
        "ISU_NM": "Name",
        "TDD_OPNPRC": "Open",
        "TDD_HGPRC": "High",
        "TDD_LWPRC": "Low",
        "TDD_CLSPRC": "Close",
        "ACC_TRDVOL": "Volume",
        "ACC_TRDVAL": "Value",
        "MKTCAP": "MarketCap",
        "LIST_SHRS": "ListShares",
    }
    df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns}, inplace=True)
    df["Market"] = market
    return df

In [5]:
def fetch_base_info(auth_key: str, bas_dd: str, market: str = "KOSPI") -> pd.DataFrame:
    market = market.upper()
    endpoint = ENDPOINTS[market]["base_info"]
    data = _krx_post(auth_key, endpoint, {"basDd": bas_dd})
    rows = data.get("OutBlock_1", [])
    df = pd.DataFrame(rows)
    df["Market"] = market
    return df

In [6]:
AUTH_KEY = "537250F73A504836870208F5AA4F0A8BE512891F"

bas_dd = "20260109"


In [7]:
# 종목기본정보
df_base = fetch_base_info(AUTH_KEY, bas_dd, market="KOSPI")
print("BASE rows:", len(df_base))
print(df_base.head())

BASE rows: 956
         ISU_CD ISU_SRT_CD      ISU_NM ISU_ABBRV                ISU_ENG_NM  \
0  KR7095570008     095570   AJ네트웍스보통주    AJ네트웍스      AJ Networks Co.,Ltd.   
1  KR7006840003     006840    AK홀딩스보통주     AK홀딩스         AK Holdings, Inc.   
2  KR7282330000     282330   BGF리테일보통주    BGF리테일                BGF Retail   
3  KR7027410000     027410      BGF보통주       BGF                       BGF   
4  KR7138930003     138930  BNK금융지주보통주   BNK금융지주  BNK Financial Group Inc.   

    LIST_DD MKT_TP_NM SECUGRP_NM SECT_TP_NM KIND_STKCERT_TP_NM PARVAL  \
0  20150821     KOSPI         주권          -                보통주   1000   
1  19990811     KOSPI         주권          -                보통주   5000   
2  20171208     KOSPI         주권          -                보통주   1000   
3  20140519     KOSPI         주권          -                보통주   1000   
4  20110330     KOSPI         주권          -                보통주   5000   

   LIST_SHRS Market  
0   45252759  KOSPI  
1   13247561  KOSPI  
2   1728390

In [8]:
# 일별매매정보(OHLCV 등)
df_trd = fetch_daily_trade(AUTH_KEY, bas_dd, market="KOSPI")
print("TRD rows:", len(df_trd))
print(df_trd[["Date", "Code", "Name", "Open", "High", "Low", "Close", "Volume"]].head())

TRD rows: 956
       Date    Code     Name    Open    High     Low   Close  Volume
0  20260109  095570   AJ네트웍스    4590    4630    4560    4620   70335
1  20260109  006840    AK홀딩스    8320    8350    8080    8080   12868
2  20260109  027410      BGF    3705    3775    3705    3775   52755
3  20260109  282330   BGF리테일  102400  104500  102250  103400   54217
4  20260109  138930  BNK금융지주   15430   15510   15180   15300  726741
